In [25]:
import numpy as np
from IPython.display import display, HTML


In [26]:
import numpy as np

def diagonal_dominant(a, b):
    matrix = np.column_stack([a, b])
    
    matrix[0], matrix[2] = matrix[2].copy(), matrix[0].copy()
    
    matrix /= matrix[:, 0].reshape((-1, 1))
    for i in range(1, 4):
        matrix[i] -= matrix[i][0] * matrix[0]

    matrix[2] += 8 * matrix[1]
    matrix[0] = matrix[0] - 11 * matrix[1] + matrix[2]
    return matrix[:, :-1], matrix[:, -1:]

def gaussian_elimination_one_step(A, b, column):
    A_copy = np.copy(A)
    b_copy = np.copy(b)
    n = len(A_copy)

    max_row = column
    for i in range(column + 1, n):
        if abs(A_copy[i][column]) > abs(A_copy[max_row][column]):
            max_row = i

    A_copy[[column, max_row]] = A_copy[[max_row, column]]
    b_copy[[column, max_row]] = b_copy[[max_row, column]]


    for i in range(column + 1, n):
        factor = A_copy[i][column] / A_copy[column][column]
        A_copy[i] -= factor * A_copy[column]
        b_copy[i] -= factor * b_copy[column]
    
    
    return A_copy, b_copy

A = np.array([
    [2.12, 0.42, 1.34, 0.88],
    [0.42, 3.95, 1.87, 0.43],
    [1.34, 1.87, 2.98, 0.46],
    [0.88, 0.43, 0.46, 4.44]
])


b = np.array([11.172, 0.115, 0.009, 9.349])


print (diagonal_dominant(A, b))

column = 0 
A_step, b_step = gaussian_elimination_one_step(A, b, column)

print("Original Matrix:")
print(A)
print("\nConstant vector:")
print(b)
print("\nMatrix after one step of Gaussian elimination:")
print(A_step)
print("\nConstant vector after one step of Gaussian elimination:")
print(b_step)

print(A_step.tolist())
print(b_step.tolist())

(array([[  1.          , -23.8296053426,  -6.0534255944,  -1.6264834855],
       [  0.          ,   8.0092395167,   2.2285003554,   0.6805259417],
       [  0.          ,  62.8765069531,  16.2361977176,   5.5160182913],
       [  0.          ,  -0.9068860244,  -1.7011533243,   4.7021709634]]), array([[ 4.4685320031],
       [ 0.2670931059],
       [ 7.39983975  ],
       [10.6171472185]]))
Original Matrix:
[[2.12 0.42 1.34 0.88]
 [0.42 3.95 1.87 0.43]
 [1.34 1.87 2.98 0.46]
 [0.88 0.43 0.46 4.44]]

Constant vector:
[11.172  0.115  0.009  9.349]

Matrix after one step of Gaussian elimination:
[[ 2.12          0.42          1.34          0.88        ]
 [ 0.            3.8667924528  1.6045283019  0.2556603774]
 [ 0.            1.6045283019  2.1330188679 -0.0962264151]
 [ 0.            0.2556603774 -0.0962264151  4.0747169811]]

Constant vector after one step of Gaussian elimination:
[11.172        -2.0983207547 -7.0525471698  4.7115660377]
[[2.12, 0.42, 1.34, 0.88], [0.0, 3.86679245283018

In [27]:
import numpy as np
from numpy import ndarray
np.set_printoptions(precision=10, suppress=True)




def jacobi(A, b, tolerance=1e-6, max_iterations=10000):
    global jacobi_b
    x = np.zeros_like(b, dtype=np.double)
    T = A - np.diag(np.diagonal(A))
    
    for k in range(max_iterations):
        print(f"Ітерація №{k+1}")
        x_old = x

        x = (b - np.dot(T, x)) / np.diagonal(A).reshape((-1, 1))
        
        print("Наближення")
        print(x.reshape((-1, 1)))
        
        print("Вектор нев'язки")
        vector = b - np.dot(A, x.reshape((-1, 1)))
        print(vector)
        
        if np.allclose(x_old, x, atol=tolerance, rtol=0.):
            break
        
        jacobi_b = vector
    
    return x

def seidel(A, b, tolerance=1e-6, max_iterations=10000):
    global seidel_b
    n = len(A)
    x = np.zeros_like(b, dtype=np.double)
    converge = False
    
    for k in range(max_iterations):
        if converge:
            break
        print(f"Ітерація №{k+1}")
        x_new = np.copy(x)
        
        for i in range(n):
            s1 = sum(A[i][j] * x_new[j] for j in range(i))
            s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i][i]
        
        print("Наближення")
        print(x_new.reshape((-1, 1)))
        
        print("Вектор нев'язки")
        vector = b - np.dot(A, x_new.reshape((-1, 1)))
        converge = np.allclose(x, x_new, atol=tolerance, rtol=0.)
        print(vector)
        
        seidel_b = vector
        x = x_new
    
    return x.reshape((-1, 1))


A_origin = np.array([
    [2.12, 0.42, 1.34, 0.88],
    [0.42, 3.95, 1.87, 0.43],
    [1.34, 1.87, 2.98, 0.46],
    [0.88, 0.43, 0.46, 4.44]
])

A = np.array([[2.12, 0.42, 1.34, 0.88], [0.0, 3.8667924528301887, 1.6045283018867926, 0.25566037735849056], [0.0, 1.6045283018867926, 2.1330188679245285, -0.09622641509433955], [0.0, 0.25566037735849056, -0.09622641509433955, 4.074716981132076]])
b = np.array([[11.172], [-2.0983207547169807], [-7.05254716981132], [4.7115660377358495]])

tolerance = 1e-6
Ab = np.column_stack([A, b])



print("="*50)
print("Матриця за варіантом завдання")
print("----------------------------------------------------------------------------------------")
print(A_origin)
print("="*50)
print("Вхідна матриця (З діагональною перевагою після ітерації методу гауса)")
print("----------------------------------------------------------------------------------------")
print(A)
print("="*50)
print("Зведена матриця")
print("----------------------------------------------------------------------------------------")
print(np.column_stack([A, b]))
print("="*50)
print("Метод Зейделя")
print("----------------------------------------------------------------------------------------")
print("Відповідь:", sei := seidel(A.copy(), b.copy(), tolerance=tolerance), sep="\n")
print("="*50)
print("Метод простих ітерацій")
print("----------------------------------------------------------------------------------------")
print("Відповідь:", jac := jacobi(A.copy(), b.copy(), tolerance=0.1), sep="\n")
print("="*50)

print(f"MSE SEIDEL: {np.mean((seidel_b) ):.5f}")
print(f"MSE Jacoby: {np.mean((jacobi_b) ):.5f}")

Матриця за варіантом завдання
----------------------------------------------------------------------------------------
[[2.12 0.42 1.34 0.88]
 [0.42 3.95 1.87 0.43]
 [1.34 1.87 2.98 0.46]
 [0.88 0.43 0.46 4.44]]
Вхідна матриця (З діагональною перевагою після ітерації методу гауса)
----------------------------------------------------------------------------------------
[[ 2.12          0.42          1.34          0.88        ]
 [ 0.            3.8667924528  1.6045283019  0.2556603774]
 [ 0.            1.6045283019  2.1330188679 -0.0962264151]
 [ 0.            0.2556603774 -0.0962264151  4.0747169811]]
Зведена матриця
----------------------------------------------------------------------------------------
[[ 2.12          0.42          1.34          0.88         11.172       ]
 [ 0.            3.8667924528  1.6045283019  0.2556603774 -2.0983207547]
 [ 0.            1.6045283019  2.1330188679 -0.0962264151 -7.0525471698]
 [ 0.            0.2556603774 -0.0962264151  4.0747169811  4.7115660